In [2]:
import pickle
import tensorflow as tf
import keras
import keras.backend as K
from keras.layers import *
from keras.models import Model
from keras.callbacks import ModelCheckpoint
from keras.models import save_model
from keras.models import load_model
import os

X = pickle.load(open("../data/X_data_random.pkl","rb"))
y = pickle.load(open("../data/y_data_random.pkl","rb"))
word2id, id2word = pickle.load(open("../data/word2idandid2word.pkl","rb"))
label2id, id2label = pickle.load(open("../data/label2idandid2label.pkl","rb"))
word2vec_matrix = pickle.load(open("../data/word2vec_matrix.pkl","rb"))


index1 = 50000
index2 = 70000
X_train, X_val, X_test = X[:index1], X[index1:index2], X[index2:]
y_train, y_val, y_test = y[:index1], y[index1:index2], y[index2:]

os.environ['CUDA_VISIBLE_DEVICES'] = "0"

padding_len = 200

with tf.device('/gpu:0'):
    model_input = Input(shape=(padding_len,))
    
    wordembed = Embedding(len(word2vec_matrix), 300, weights=[word2vec_matrix], input_length=padding_len, trainable=False)(model_input)
    sen2vec = Bidirectional(GRU(300, activation='tanh', return_sequences=True))(wordembed)
    attention = Dense(1, activation='tanh')(sen2vec)
    attention = Flatten()(attention)
    attention = Activation('softmax')(attention)
    attention = RepeatVector(300*2)(attention)
    attention = Permute([2, 1])(attention)
    sent_representation = Multiply()([sen2vec, attention])
    convs = []
    filter_size = [2, 3, 4, 5]
    for i in filter_size:
        conv_layer = Conv1D(filters=128, kernel_size=i, activation='relu')(sent_representation)
        pool_layer = MaxPooling1D(200-i+1, 1)(conv_layer)
        pool_layer = Flatten()(pool_layer)
        convs.append(pool_layer)
    sen2vec_conv = concatenate(convs, axis=1)
    sen2vec_conv = Dropout(0.5)(sen2vec_conv)
    model_output = Dense(len(label2id), activation='softmax')(sen2vec_conv)
    model = Model(inputs=model_input, outputs=model_output)
    model.summary()
    checkpoint = ModelCheckpoint("../PCmodel/BiGRUTextCNNAttention.h5", monitor='val_categorical_accuracy', verbose=1, save_best_only=True, mode='max')
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['categorical_accuracy'])
    model.fit(X_train, y_train, batch_size=200, epochs=20, validation_data=(X_val, y_val), callbacks=[checkpoint])
    model.save("../PCmodel/BiGRUTextCNNAttention.h5")

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 200)          0                                            
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, 200, 300)     68857800    input_2[0][0]                    
__________________________________________________________________________________________________
bidirectional_2 (Bidirectional) (None, 200, 600)     1081800     embedding_2[0][0]                
__________________________________________________________________________________________________
dense_2 (Dense)                 (None, 200, 1)       601         bidirectional_2[0][0]       

Epoch 8/20
50000/50000 [==============================] - 120s 2ms/step - loss: 0.6357 - categorical_accuracy: 0.7812 - val_loss: 0.6200 - val_categorical_accuracy: 0.7808

Epoch 00008: val_categorical_accuracy improved from 0.77055 to 0.78085, saving model to ../PCmodel/BiGRUTextCNN.h5
Epoch 9/20
50000/50000 [==============================] - 120s 2ms/step - loss: 0.6038 - categorical_accuracy: 0.7902 - val_loss: 0.6024 - val_categorical_accuracy: 0.7857

Epoch 00009: val_categorical_accuracy improved from 0.78085 to 0.78575, saving model to ../PCmodel/BiGRUTextCNN.h5
Epoch 10/20
50000/50000 [==============================] - 120s 2ms/step - loss: 0.5725 - categorical_accuracy: 0.8003 - val_loss: 0.5886 - val_categorical_accuracy: 0.7906

Epoch 00010: val_categorical_accuracy improved from 0.78575 to 0.79055, saving model to ../PCmodel/BiGRUTextCNN.h5
Epoch 11/20
50000/50000 [==============================] - 120s 2ms/step - loss: 0.5480 - categorical_accuracy: 0.8090 - val_loss: 0.56

In [3]:
y_pred = model.predict(X_test)
y1 = [np.argmax(i) for i in y_test]
y2 = [np.argmax(i) for i in y_pred]

from sklearn import metrics

print('precision:')
print(metrics.precision_score(y1,y2,average='micro'))
print(metrics.precision_score(y1,y2,average='macro'))

print('recall:')
print(metrics.recall_score(y1,y2,average='micro'))
print(metrics.recall_score(y1,y2,average='macro'))

print('f1:')
print(metrics.f1_score(y1,y2,average='micro'))
print(metrics.f1_score(y1,y2,average='macro'))

precision:
0.8109
0.8110388404274043
recall:
0.8109
0.8094290705104031
f1:
0.8109
0.8092023096591756
